<a href="https://colab.research.google.com/github/lianghunan17/python/blob/master/%E2%97%8B%E4%BB%98%E3%81%8D%E7%A2%BA%E8%AA%8D.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [95]:
import openpyxl
from openpyxl.styles import PatternFill, Alignment
from openpyxl.utils import get_column_letter
import openpyxl as xl
from openpyxl.styles.borders import Border, Side
# ここから枠線の設定
from openpyxl.styles import Border, Side
# openpyxl から必要なモジュールをインポート
from openpyxl.styles import Font, Color

# Excel ファイルを開く
wb = openpyxl.load_workbook('/content/データ仕分けツール .xlsm')

# "データ_仕分け" シートを取得
data_sheet = wb['データ_仕分け']

# B8 セルから始まるB列の全てのシート名を取得
sheet_names = [cell[0].value for cell in data_sheet['B8':f'B{data_sheet.max_row}'] if cell[0].value is not None]

# 新しいWorkbookを作成
new_wb = openpyxl.Workbook()

# 最初のSheetを削除
default_sheet = new_wb['Sheet']
new_wb.remove(default_sheet)

# 各シート名に対して処理を実行
for sheet_name in sheet_names:
    sheet = wb[sheet_name.strip()]  # シート名の前後の空白を削除

    # D列のユニークな値を取得
    unique_D_values = list(set([cell.value for cell in sheet['D'] if cell.value is not None]))

    # C列のユニークな値を取得
    unique_C_values = list(set([cell.value for cell in sheet['C'] if cell.value is not None]))
    # 新しいシートのデータの文字を太字にするため、新たなフォントスタイルを作成します。
    bold_font = Font(bold=True,color='FF0000')
    # 黄色のパターンフィルを作成
    yellow_fill = PatternFill(start_color='FFFF00', end_color='FFFF00', fill_type='solid')
    # 新しいシートを作成
    new_sheet = new_wb.create_sheet(title=f"{sheet_name.strip()}〇付き確認")
    new_sheet['A1'] = sheet_name.strip()
    new_sheet['A1'].alignment = Alignment(horizontal="center")
    new_sheet['A1'].fill= yellow_fill
    new_sheet['A1'].font = openpyxl.styles.fonts.Font(color='FF0000')
    new_sheet['A1'].font = bold_font

    # 新たな塗りつぶしパターンを作成します。
    aqua_fill = PatternFill(start_color="9DC3E6", end_color="9DC3E6", fill_type="solid")
 

# 別の塗りつぶしパターンを作成します。
    olive_fill = PatternFill(start_color="E8F5E9", end_color="E8F5E9", fill_type="solid")
# 新しいシートのデータの文字を太字にするため、新たなフォントスタイルを作成します。
    bold_font = Font(bold=True)

    # 新しいシートにD列のユニークな値をB2から横並びに張り付ける
    for i, value in enumerate(unique_D_values, start=2):
        new_sheet.cell(row=2, column=i, value=value)
        new_sheet.cell(row=2, column=i).fill = aqua_fill
        new_sheet.cell(row=2, column=i).font = bold_font 
           
# set column width
        for col in new_sheet.columns:
              max_length = 0
              column = col[0].column
              column_letter = get_column_letter(col[0].column)
      
              for cell in col:
                                       
                  if len(str(cell.value)) > max_length:
                   max_length = len(str(cell.value))
                   adjusted_width = (max_length + 6)* 1.5
                   new_sheet.column_dimensions[column_letter].width = adjusted_width
        

    # 新しいシートにC列のユニークな値を張り付ける
    for i, unique_C_value in enumerate(unique_C_values, start=3):

        new_sheet.cell(row=i, column=1, value=unique_C_value)
        new_sheet.cell(row=i, column=1).fill = olive_fill
        new_sheet.cell(row=i, column=1).font = bold_font 
        new_sheet.column_dimensions['A'].width =26       
           


    # 枠線の設定を作成
    thin_border = Border(left=Side(style='thin', color='000000'), 
                     right=Side(style='thin', color='000000'), 
                     top=Side(style='thin', color='000000'), 
                     bottom=Side(style='thin', color='000000'))

    # データ_仕分けシートでB2から始まる行値とA列値を参照して検索し、該当する場合は新しいシートに〇をつける
    for row in sheet.iter_rows(min_row=2, min_col=1, max_col=sheet.max_column, values_only=True):
        for i, unique_D_value in enumerate(unique_D_values, start=2):
            for j, unique_C_value in enumerate(unique_C_values, start=3):
                if row[3] == unique_D_value and row[2] == unique_C_value:  # Excelの列は0からではなく1から始まるので注意
                    new_sheet.cell(row=j, column=i, value="〇")
                    new_sheet.cell(row=j, column=i).fill = yellow_fill
                   
    
    # データをすべて中央に揃える
    for row in new_sheet.iter_rows():
        for cell in row:
            cell.alignment = Alignment(horizontal="center")
            cell.border = thin_border

    
    # セルrange("A2")にデータを追加
    new_sheet['A2'].value = 'ー'
   
    # 新しいシートの行2と3にフィルターを付ける
    new_sheet.auto_filter.ref = f"A2:{get_column_letter(new_sheet.max_column)}3"
       
    # ウインドウ枠の固定 (B2)
    new_sheet.freeze_panes = new_sheet['B2']

# 新しいWorkbookを保存
new_wb.save('/content/○付き確認.xlsx')




